In [ ]:
import pandas as pd
def load_info():
    SignId_to_ShapeId,ClassId_to_SignId={},{}
    df=pd.read_csv("data/Meta.csv")
    df=df[["ClassId","ShapeId","SignId"]]
    df_filled = df.fillna("0")
    for row in df_filled.itertuples():
        ClassId_to_SignId[row.ClassId]=row.SignId
        if row.SignId not in SignId_to_ShapeId:
            SignId_to_ShapeId[row.SignId]=row.ShapeId
        else:
            if SignId_to_ShapeId[row.SignId]!=row.ShapeId:
                print("SignId to ShapeId is not a function!")
    print("SignId to ShapeId is a function!")
    return (SignId_to_ShapeId,ClassId_to_SignId)

SignId_to_ShapeId,ClassId_to_SignId=load_info()


SignId to ShapeId is a function!


In [12]:
import cv2
import os
import pandas as pd

IMG_WIDTH=30
IMG_HEIGHT=30

# a method suitable for both train data and test data loading

def load_data(data_dir, csv_file_path):
    images,labels=[],[]
    df=pd.read_csv(csv_file_path)
    df=df[["ClassId","Path"]]
    for row in df.itertuples():
        image_path=os.path.join(data_dir, row.Path)
        image = cv2.imread(image_path)
        resized_image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        images.append(resized_image)
        labels.append(int(ClassId_to_SignId[row.ClassId]))
        '''print(images)
        print(type(images[0]))
        print(images[0].shape)
        print(labels)'''
    print("Data loaded")
    return (images,labels)

# loading the test and training data
def load_train_and_test_data(data_dir):
    train,test="Train.csv","Test.csv"
    files=set([f for f in os.listdir(data_dir)])
    if test in files:
        test_label_path=os.path.join(data_dir,test)
        x_train,y_train=load_data(data_dir,test_label_path)
    else:
        raise Exception("Test labels are not available")
    if train in files:
        train_label_path=os.path.join(data_dir,train)
        x_test,y_test=load_data(data_dir,train_label_path)
    else:
        raise Exception("Training labels are not available")
    return (x_train,y_train,x_test,y_test)

In [13]:
import numpy as np

def data_preprocessing(x_train,x_test):
    # mean subtraction and normalization
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)
    x_train = (x_train - mean) / std
    x_test = (x_test - mean) / std
    print("Data preprocessing completed.")
    return (x_train,x_test)

In [14]:
import tensorflow as tf
import numpy as np
import sys
import pandas as pd
from model_training import get_model
from data_preprocessing import data_preprocessing
from data_loading import load_train_and_test_data

EPOCHS = 10
num_of_categories=43
IMG_WIDTH=30
IMG_HEIGHT=30

# load the train and test data
x_train, y_train, x_test, y_test = load_train_and_test_data("data")
x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
# one hot encoding
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
x_train, x_test=data_preprocessing(x_train, x_test)

Data loaded
Data loaded
Data preprocessing completed.


In [22]:
import tensorflow as tf
num_of_categories=43
IMG_WIDTH=30
IMG_HEIGHT=30
regularizer_strength=0.3
dropout_rate=0.3
def get_model(optimizer):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(regularizer_strength), input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(regularizer_strength), kernel_initializer='he_normal'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation="relu", kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(regularizer_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.Dense(num_of_categories, activation="softmax")
    ])
    model.compile(
        optimizer=optimizer,  
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    print("Model trained.")
    return model

In [23]:
# evaluate the adam performance
model = get_model("adam")

model.fit(x_train, y_train, epochs=EPOCHS, batch_size=32)

loss,accuracy = model.evaluate(x_test, y_test, verbose=2, batch_size=32)

print("the accuracy with nadam optimizer is ", accuracy)

# evaluate the nadam performance, expected to be close to the above
model = get_model("nadam")

model.fit(x_train, y_train, epochs=EPOCHS, batch_size=32)

loss,accuracy = model.evaluate(x_test, y_test, verbose=2, batch_size=32)

print("the accuracy with adam optimizer is ", accuracy)



Model trained.
Epoch 1/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 133s 241ms/step - accuracy: 0.4937 - loss: 37.8236
Epoch 2/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 113s 166ms/step - accuracy: 0.6924 - loss: 5.1687
Epoch 3/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 97s 243ms/step - accuracy: 0.7568 - loss: 3.6989
Epoch 4/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 130s 211ms/step - accuracy: 0.8050 - loss: 2.7798
Epoch 5/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 153s 237ms/step - accuracy: 0.8331 - loss: 2.2542
Epoch 6/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 95s 238ms/step - accuracy: 0.8506 - loss: 1.9856
Epoch 7/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 137s 222ms/step - accuracy: 0.8539 - loss: 1.7893
Epoch 8/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 127s 181ms/step - accuracy: 0.8757 - loss: 1.5527
Epoch 9/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 89s 198ms/step - accuracy: 0.8657 - loss: 1.5399
Epoch 10/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 86s 206ms/step - accuracy: 0.8795 - loss: 1.4292
1226/1226 - 67s - 54ms/step - accuracy: 0.7757 - loss: 1.7374
the accuracy with nadam o

MemoryError: 